In [1]:
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook
 
ib = IB()
ib.connect('127.0.0.1', 7496, clientId=2)
print(ib.isConnected())

API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 7496)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 111] Connect call failed ('127.0.0.1', 7496)

In [6]:
import os
import numpy as np
import pandas as pd
from math import atan2, pi
import datetime
from datetime import timedelta
from scipy.stats import linregress
from statistics import stdev

contract = Stock('AAPL', 'SMART', 'USD', primaryExchange='NASDAQ')

def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, intercept, rvalue, _, _ = linregress(x, returns)
    momo = ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2
    momo_price = np.e ** (intercept + slope*x)
    return  momo

def atr(df, n=20):
    data = df.copy()
    high = data.high
    low = data.low
    close = data.close
    dopen = data.open
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = tr.ewm(alpha=1/n, min_periods=n, adjust=False).mean() #J. Welles Wilder's EMA 
    tr = close - dopen    # home run day range
    return tr, atr

def angle(df):
    #return np.rad2deg(np.arctan2(df['average']-df['average'].shift(period-1), period-1))
    return np.rad2deg(np.arctan2(df['average'].shift(2)-df['average'].shift(1), 1))


def reqHistData(contract, endDateTime, durationStr, barSizeSetting):
    extend_hour = False
    
    if 'day' in barSizeSetting: #only use intraday data
        extend_hour = True
    
    bars = ib.reqHistoricalData(contract, endDateTime=endDateTime, durationStr=durationStr,
        barSizeSetting=barSizeSetting, whatToShow='TRADES', useRTH=extend_hour)
    df = util.df(bars)
    
    if df is None:
        return df
    
    df.drop(columns=['barCount'], inplace =True)

    if (
        'hours' in barSizeSetting or 
        'mins' in barSizeSetting or 
        'secs' in barSizeSetting
    ):
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df['day'] = df.date.dt.day
        df['hour'] = df.date.dt.hour
        df['minute'] = df.date.dt.minute
    else:
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df.date.dt.year
        df['month'] = df.date.dt.month
        df['day'] = df.date.dt.day
        
    df.reset_index()
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df =  df.sort_index(ascending=True)
    
    if '1 day' in barSizeSetting:
        df['gap'] = 100 * (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
        df['avg50_volume'] = df['volume'].rolling(50).mean()
        #df['avg50_volume'].fillna(0, inplace=True)
        df['angle_2d'] = angle(df)
        #df['momo'], df['momo_price'] = zip(*df['close'].rolling(90).apply(momentum, raw=False))
        df['momo_90d'] = df['close'].rolling(90).apply(momentum, raw=False)
        df['tr'], df['atr_20d'] = atr(df, 20)
        df['ema21'] = df['close'].ewm(span=21,adjust=True,ignore_na=False).mean() 
        df['ema50'] = df['close'].ewm(span=50,adjust=True,ignore_na=False).mean()
        df['ema200'] = df['close'].ewm(span=200,adjust=True,ignore_na=False).mean()
        df['stdev50_volume'] = stdev(df['volume'], 50)
    
    #print(df.info())
    #print(df.head())
    return df



start = '20191231 20:00:00'

def downloader(contract, start, end, durationStr='1 Y', barSizeSetting='1 day'):
    substr = '1231 20:00:00'
    df = pd.DataFrame()
    
    while True:
        endDateTime = str(start) + substr
        
        print("processing: " + endDateTime)
        df_sub = reqHistData(contract, endDateTime=endDateTime, durationStr=durationStr, barSizeSetting=barSizeSetting)
        df = pd.concat([df_sub, df])
        df = df.loc[~df.index.duplicated(keep='first')]
        
        start -= 1
        if start < end:
            print("done")
            break
            
    print(df.info())        
    return df
            


In [7]:
# download daily
df = reqHistData(contract, endDateTime=start, durationStr='28 Y', barSizeSetting='1 day')
 
file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_daily' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

In [8]:
df.head()

open   high    low  close  volume   average  year  month  day  \
date                                                                         
1992-01-08  8.355  8.750  8.355  8.645  145467  8.642857  1992      1    8   
1992-01-09  8.645  8.895  8.605  8.895  130319  8.892857  1992      1    9   
1992-01-10  8.785  8.930  8.715  8.895  122640  8.892857  1992      1   10   
1992-01-13  8.895  8.965  8.785  8.855   67410  8.857143  1992      1   13   
1992-01-14  8.895  9.250  8.895  9.215  171129  9.214286  1992      1   14   

                 gap  avg50_volume   angle_2d  momo_90d    tr  atr_20d  \
date                                                                     
1992-01-08       NaN           NaN        NaN       NaN  0.29      NaN   
1992-01-09  0.000000           NaN        NaN       NaN  0.25      NaN   
1992-01-10 -1.236650           NaN -14.036243       NaN  0.11      NaN   
1992-01-13  0.000000           NaN   0.000000       NaN -0.04      NaN   
1992-01-14  0.451722           NaN   2.045408       NaN  0.32      NaN   

               ema21     ema50    ema200  stdev50_volume  
date                                                      
1992-01-08  8.645000  8.645000  8.645000   831647.744559  
1992-01-09  8.775952  8.772500  8.770625   831647.744559  
1992-01-10  8.819471  8.814977  8.812499   831647.744559  
1992-01-13  8.829661  8.825591  8.823284   831647.744559  
1992-01-14  8.922071  8.909826  8.903202   831647.744559

Peer closed connection


In [18]:
# download 4h
df = reqHistData(contract, endDateTime=start, durationStr='30 Y', barSizeSetting='4 hours')
 
file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_4h' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16771 entries, 2004-01-23 08:00:00 to 2019-12-31 19:00:00
Data columns (total 11 columns):
open       16771 non-null float64
high       16771 non-null float64
low        16771 non-null float64
close      16771 non-null float64
volume     16771 non-null int64
average    16771 non-null float64
year       16771 non-null int64
month      16771 non-null int64
day        16771 non-null int64
hour       16771 non-null int64
minute     16771 non-null int64
dtypes: float64(5), int64(6)
memory usage: 1.5 MB
None


In [11]:
df = downloader(contract, 2019, 1990, '1 Y', '4 hours')

file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_4h' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

processing: 20191231 20:00:00
processing: 20181231 20:00:00
processing: 20171231 20:00:00
processing: 20161231 20:00:00
processing: 20151231 20:00:00
processing: 20141231 20:00:00
processing: 20131231 20:00:00
processing: 20121231 20:00:00
processing: 20111231 20:00:00
processing: 20101231 20:00:00
processing: 20091231 20:00:00
processing: 20081231 20:00:00
processing: 20071231 20:00:00
processing: 20061231 20:00:00
processing: 20051231 20:00:00
processing: 20041231 20:00:00


Error 162, reqId 20: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20031231 20:00:00
processing: 20021231 20:00:00


Error 162, reqId 21: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 22: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 23: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')

processing: 20011231 20:00:00
processing: 20001231 20:00:00


...

Error 162, reqId 29: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 30: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 31: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')

processing: 19931231 20:00:00
processing: 19921231 20:00:00


processing: 19911231 20:00:00


Error 162, reqId 32: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 33: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 19901231 20:00:00
done
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16771 entries, 2004-01-23 08:00:00 to 2019-12-31 19:00:00
Data columns (total 11 columns):
open       16771 non-null float64
high       16771 non-null float64
low        16771 non-null float64
close      16771 non-null float64
volume     16771 non-null int64
average    16771 non-null float64
year       16771 non-null int64
month      16771 non-null int64
day        16771 non-null int64
hour       16771 non-null int64
minute     16771 non-null int64
dtypes: float64(5), int64(6)
memory usage: 1.5 MB
None


In [12]:
# download 10m

df = downloader(contract, 2019, 1990, '1 Y', '10 mins')

file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_10m' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

processing: 20191231 20:00:00
processing: 20181231 20:00:00
processing: 20171231 20:00:00
processing: 20161231 20:00:00
processing: 20151231 20:00:00
processing: 20141231 20:00:00
processing: 20131231 20:00:00
processing: 20121231 20:00:00
processing: 20111231 20:00:00
processing: 20101231 20:00:00
processing: 20091231 20:00:00
processing: 20081231 20:00:00
processing: 20071231 20:00:00
processing: 20061231 20:00:00
processing: 20051231 20:00:00
processing: 20041231 20:00:00
processing: 20031231 20:00:00


Error 162, reqId 50: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20021231 20:00:00


Error 162, reqId 51: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20011231 20:00:00


...

processing: 19931231 20:00:00
processing: 19921231 20:00:00


Error 162, reqId 61: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 62: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 19911231 20:00:00
processing: 19901231 20:00:00


Error 162, reqId 63: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


done
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 353920 entries, 2004-01-23 09:30:00 to 2019-12-31 19:50:00
Data columns (total 11 columns):
open       353920 non-null float64
high       353920 non-null float64
low        353920 non-null float64
close      353920 non-null float64
volume     353920 non-null int64
average    353920 non-null float64
year       353920 non-null int64
month      353920 non-null int64
day        353920 non-null int64
hour       353920 non-null int64
minute     353920 non-null int64
dtypes: float64(5), int64(6)
memory usage: 32.4 MB
None


In [3]:
# download 30m

df = downloader(contract, 2019, 1990, '1 Y', '30 mins')

file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_30m' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

processing: 20191231 20:00:00
processing: 20181231 20:00:00
processing: 20171231 20:00:00
processing: 20161231 20:00:00
processing: 20151231 20:00:00
processing: 20141231 20:00:00
processing: 20131231 20:00:00
processing: 20121231 20:00:00
processing: 20111231 20:00:00
processing: 20101231 20:00:00
processing: 20091231 20:00:00
processing: 20081231 20:00:00
processing: 20071231 20:00:00
processing: 20061231 20:00:00
processing: 20051231 20:00:00
processing: 20041231 20:00:00


Error 162, reqId 19: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20031231 20:00:00
processing: 20021231 20:00:00


Error 162, reqId 20: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 21: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20011231 20:00:00
processing: 20001231 20:00:00


...

processing: 19931231 20:00:00
processing: 19921231 20:00:00


Error 162, reqId 31: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 19911231 20:00:00
processing: 19901231 20:00:00


Error 162, reqId 32: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


done
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 118623 entries, 2004-01-23 09:30:00 to 2019-12-31 19:30:00
Data columns (total 11 columns):
open       118623 non-null float64
high       118623 non-null float64
low        118623 non-null float64
close      118623 non-null float64
volume     118623 non-null int64
average    118623 non-null float64
year       118623 non-null int64
month      118623 non-null int64
day        118623 non-null int64
hour       118623 non-null int64
minute     118623 non-null int64
dtypes: float64(5), int64(6)
memory usage: 10.9 MB
None


In [13]:
# download 5m 706614
df = downloader(contract, 2019, 1990, '1 Y', '5 mins')

file = str(contract.symbol) + '_' + str(df.iloc[0].year.astype(int)) + '_' + str(df.iloc[-1].year.astype(int)) + '_5m' + '.csv'
path = "E:\Day Trading\Historical Data"
output_file = os.path.join(path, file)
export_csv = df.to_csv(output_file, header=True)

processing: 20191231 20:00:00
processing: 20181231 20:00:00
processing: 20171231 20:00:00
processing: 20161231 20:00:00
processing: 20151231 20:00:00
processing: 20141231 20:00:00
processing: 20131231 20:00:00
processing: 20121231 20:00:00
processing: 20111231 20:00:00
processing: 20101231 20:00:00
processing: 20091231 20:00:00
processing: 20081231 20:00:00
processing: 20071231 20:00:00
processing: 20061231 20:00:00
processing: 20051231 20:00:00
processing: 20041231 20:00:00


Error 162, reqId 80: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20031231 20:00:00


Error 162, reqId 81: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 20021231 20:00:00


...

processing: 19931231 20:00:00
processing: 19921231 20:00:00


Error 162, reqId 91: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
Error 162, reqId 92: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 19911231 20:00:00


Error 162, reqId 93: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')


processing: 19901231 20:00:00
done
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 706614 entries, 2004-01-23 09:30:00 to 2019-12-31 19:55:00
Data columns (total 11 columns):
open       706614 non-null float64
high       706614 non-null float64
low        706614 non-null float64
close      706614 non-null float64
volume     706614 non-null int64
average    706614 non-null float64
year       706614 non-null int64
month      706614 non-null int64
day        706614 non-null int64
hour       706614 non-null int64
minute     706614 non-null int64
dtypes: float64(5), int64(6)
memory usage: 64.7 MB
None


In [ ]:
# download 1m

In [6]:
ib.disconnect()
print(ib.isConnected())

False


In [8]:
df = reqHistData(contract, endDateTime='', durationStr='5 D', barSizeSetting='1 min', keepUpToDate=True)
print(df.tail(10))

TypeError: reqHistData() got an unexpected keyword argument 'keepUpToDate'